### Workup KIF results - part 2

Compare per interaction/feature scores across systems using a multiple sequence alingment (MSA). Make pymol projections for the per feature/interaction scores. 

In [1]:
import re
import pandas as pd
from key_interactions_finder.pymol_projections import project_pymol_top_features

KIF_FILES = {
    "TEM1": 
        {
            "benzyl": r"outputs/TEM1_1M40_benzyl/Linear_Correlations_Per_Feature_Scores.csv",
            "cefo": r"outputs/TEM1_1M40_cefo/Linear_Correlations_Per_Feature_Scores.csv"
        },
    "ENCA": 
        {
            "benzyl": r"outputs/ENCA_3ZDJ_benzyl/Linear_Correlations_Per_Feature_Scores.csv",
            "cefo": r"outputs/ENCA_3ZDJ_cefo/Linear_Correlations_Per_Feature_Scores.csv"
        },
    "GNCA": 
        {
            "benzyl": r"outputs/GNCA_4B88_benzyl/Linear_Correlations_Per_Feature_Scores.csv",
            "cefo": r"outputs/GNCA_4B88_cefo/Linear_Correlations_Per_Feature_Scores.csv"
        },
    "PNCA": 
        {
            "benzyl": r"outputs/PNCA_4C6Y_benzyl/Linear_Correlations_Per_Feature_Scores.csv",
            "cefo": r"outputs/PNCA_4C6Y_cefo/Linear_Correlations_Per_Feature_Scores.csv"
        }
}
ALINGMENT_FILE = r"raw_data/align1d.ali"

GENERALISTS = ["GNCA", "PNCA"]
SPECIALISTS = ["TEM1", "ENCA"]

helper functions to be used throughout notebook

In [2]:
def parse_fasta(file_path: str) -> dict[str, str]:
    """
    Takes a multiple sequence alingment (msa) file  produced by modeller
    (with fasta formating) and outputs a sequence as string with '-' 
    for the gaps introduced by the msa.

    Parameters
    ----------
    file_path: str
        File path to the multiple sequence alingment file.

    Returns
    ----------
    dict[int, int]
        key is the pdb residue number, value is the msa residue number.
    """
    sequences = {}
    current_name = None
    current_sequence = []

    with open(file_path, "r", encoding="utf-8") as file:
        for line in file:
            line = line.strip()

            if line.startswith(">"):
                if current_name is not None:
                    sequences[current_name] = "".join(current_sequence)
                    current_sequence = []

                current_name = line[1:]
            else:
                current_sequence.append(line)

        # Add the last sequence to the dictionary
        if current_name is not None:
            sequences[current_name] = "".join(current_sequence)

    return sequences

def create_pdb_to_msa_converter(msa_sequence: str) -> dict[int, int]:
    """
    Create a dictionary that can enable easily converting
    between pdb and msa numbers given an msa_sequence.

    Parameters
    ----------
    msa_sequence: str
        msa sequence of the target protein.

    Returns
    ----------
    dict[int, int]
        key is the pdb residue number, value is the msa residue number.
    """
    curr_msa_number, curr_pdb_numb = 0, 0
    index_pdb_msa = {}
    for msa_residue in msa_sequence:
        if msa_residue == "*":
            continue
        if msa_residue == "-":
            curr_msa_number += 1
        else:
            curr_msa_number += 1
            curr_pdb_numb += 1

            index_pdb_msa[curr_pdb_numb] = curr_msa_number
    return index_pdb_msa

def modify_column_residue_numbers(interaction_names: list[str], constant_to_add: int = 1) ->list[str]:
    """
    Modify the residue numbers in a list of interactions by adding a constant value to them. 
    default = 1. 
    """
    updated_names = []
    for column_name in interaction_names:
        res_split = re.split(r"(\d+)", column_name)

        res1_numb = int(res_split[1])
        res1_name = res_split[2]
        res2_numb = int(res_split[3])
        remainder = res_split[4]

        res1_numb += constant_to_add
        res2_numb += constant_to_add

        updated_name = str(res1_numb) + res1_name + \
            str(res2_numb) + remainder
        updated_names.append(updated_name)

    return updated_names

load in the msa data

In [3]:
msa_seqs = parse_fasta(ALINGMENT_FILE)

msa_converters = {
    "TEM1": create_pdb_to_msa_converter(msa_seqs["1M40_TEM-1"]),
    "ENCA": create_pdb_to_msa_converter(msa_seqs["3ZDJ_ENCA"]),
    "GNCA": create_pdb_to_msa_converter(msa_seqs["4B88_GNCA"]),
    "PNCA": create_pdb_to_msa_converter(msa_seqs["4C6Y_PNCA"])
}

prepare dataframes of each system

In [4]:
all_dfs = {}
for system, substrates in KIF_FILES.items():
    benzyl_df = pd.read_csv(substrates["benzyl"])
    cefo_df = pd.read_csv(substrates["cefo"])

    # merge on feature name, calc delta
    merged_df = benzyl_df.merge(cefo_df, how="outer", on="Feature")
    merged_df.columns = ["Feature", "Score_Benzyl", "Score_Cefo"]
    merged_df = merged_df.fillna(0.0)

    merged_df["Absolute Delta"] = abs(merged_df["Score_Benzyl"] - merged_df["Score_Cefo"])
    merged_df = merged_df.sort_values("Absolute Delta", ascending=False)

    # get scores and output as pymol results files
    delta_dict = dict(zip(
        merged_df["Feature"].values,
        merged_df["Absolute Delta"].values
    ))
    
    all_dfs[system] = merged_df

In [5]:
all_dfs["TEM1"]

,Feature,Score_Benzyl,Score_Cefo,Absolute Delta
21,105Ser 209Lys Hbond,-0.285994,0.379897,0.665891
36,112Leu 116Thr Hbond,-0.248764,0.278501,0.527266
32,18Arg 40Arg Hbond,-0.253587,0.265159,0.518746
398,111Asn 141Glu Hbond,-0.009367,-0.519496,0.510129
227,132Asp 167Lys Saltbr,-0.064222,0.445359,0.509580
...,...,...,...,...
402,247Glu 250Arg Saltbr,0.007732,0.006616,0.001116
332,112Leu 115Thr Hbond,-0.026361,-0.025539,0.000823
34,199Ala 253Ala Hydrophobic,-0.251479,-0.252191,0.000712
253,251Gln 254Glu Hbond,-0.054056,-0.054744,0.000688


In [6]:
msa_converter = msa_converters["TEM1"]
interaction_name = all_dfs["TEM1"]["Feature"][0]
interaction_name

'43Met 46Thr Hbond'

In [7]:
def pdb_feats_to_msa_feats(int_names: list[str], msa_converter: dict[int, int]) -> list[str]:
    """
    Convert a list of KIF labelled interaction names to have msa numbering.
    The dictionary that converts between the numbers needs to be provided. 
    """
    msa_names = []
    for int_name in int_names:
        int_split = re.split(r"(\d+)", int_name)
        res1_numb, res2_numb = int(int_split[1]), int(int_split[3])

        msa_res1, msa_res2 = msa_converter[res1_numb], msa_converter[res2_numb]
        updated_name = str(msa_res1) + int_split[2] + str(msa_res2) + int_split[4]
    
        msa_names.append(updated_name)
    return msa_names

In [8]:
for system, df in all_dfs.items(): 
    msa_converter = msa_converters[system]
    int_names = list(df["Feature"])
    msa_names = pdb_feats_to_msa_feats(int_names=int_names, msa_converter=msa_converter)
    df["Feature, MSA numbered"] = msa_names

In [9]:
all_dfs["TEM1"].head(3)

,Feature,Score_Benzyl,Score_Cefo,Absolute Delta,"Feature, MSA numbered"
21,105Ser 209Lys Hbond,-0.285994,0.379897,0.665891,107Ser 211Lys Hbond
36,112Leu 116Thr Hbond,-0.248764,0.278501,0.527266,114Leu 118Thr Hbond
32,18Arg 40Arg Hbond,-0.253587,0.265159,0.518746,20Arg 42Arg Hbond


### Now Calculate the delta between generalists and specialists

In order to project on TEM1, we need to make sure the interaction exists in TEM1 and only use those interactions.

In [10]:
SPECIALISTS, GENERALISTS

(['TEM1', 'ENCA'], ['GNCA', 'PNCA'])

In [11]:
tem1_msa_interactions = list(all_dfs["TEM1"]["Feature, MSA numbered"])
tem1_pdb_interactions = list(all_dfs["TEM1"]["Feature"])
tem1_msa_interactions[0:2]

['107Ser 211Lys Hbond', '114Leu 118Thr Hbond']

In [12]:
delta_benzyl_scores, delta_cefo_scores = {}, {}
for msa_interaction in tem1_msa_interactions:
    benzyl_special_sum, benzyl_general_sum = 0.0, 0.0
    cefo_special_sum, cefo_general_sum = 0.0, 0.0
    for system, df in all_dfs.items():
        try:
            row = df[df["Feature, MSA numbered"] == msa_interaction]
            benzyl_score = row["Score_Benzyl"].values[0]
            cefo_score = row["Score_Cefo"].values[0]
        except IndexError:
            # couldn't be found, set to 0. 
            benzyl_score, cefo_score = 0, 0
        if system in SPECIALISTS:
            benzyl_special_sum += benzyl_score
            cefo_special_sum += cefo_score
        else:
            benzyl_general_sum += benzyl_score
            cefo_general_sum += cefo_score

    delta_benzyl_scores[msa_interaction] = benzyl_special_sum - benzyl_general_sum
    delta_cefo_scores[msa_interaction] = cefo_special_sum - cefo_general_sum

Update the interaction numbering to reflect that in a TEM1 PDB file (i.e., not the simulation numbering)

In [13]:
tem1_1m40_numbering = modify_column_residue_numbers(
    interaction_names=tem1_pdb_interactions, 
    constant_to_add=25
)

In [14]:
delta_df = pd.DataFrame([delta_benzyl_scores, delta_cefo_scores]).T.reset_index()
delta_df.columns = ["Feature, MSA numbered", "delta_benzyl", "delta_cefo"]
delta_df.insert(loc=0, column="Feature, PDB numbered", value=tem1_pdb_interactions)
delta_df.insert(loc=1, column="Feature, 1M40 numbered", value=tem1_1m40_numbering)
delta_df["Max_diff"] = delta_df[["delta_benzyl", "delta_cefo"]].abs().max(axis=1)
delta_df = delta_df.sort_values(by="Max_diff", ascending=False)
delta_df.head(3)

,"Feature, PDB numbered","Feature, 1M40 numbered","Feature, MSA numbered",delta_benzyl,delta_cefo,Max_diff
129,43Met 46Thr Hbond,68Met 71Thr Hbond,45Met 48Thr Hbond,-0.890221,-0.682554,0.890221
255,139Arg 146Glu Saltbr,164Arg 171Glu Saltbr,141Arg 148Glu Saltbr,0.698251,0.777367,0.777367
13,138Asp 153Arg Saltbr,163Asp 178Arg Saltbr,140Asp 155Arg Saltbr,-0.399970,-0.722681,0.722681


Save top interaction differences for an SI table

In [15]:
top_deltas_df = delta_df.head(40).round(2).drop(["Feature, PDB numbered", "Feature, MSA numbered", "Max_diff"], axis=1)
top_deltas_df.columns = ["Interaction", "Delta Benzyl Penicillin", "Delta Cefotaxime"]
top_deltas_df.to_csv(r"outputs/delta_special_general/max_per_interaction_diffs.csv", index=False)

### Make pymol projections (onto TEM1) of these results

In [16]:
# get scores and output as pymol results files
benzyl_dict = dict(zip(
    delta_df["Feature, PDB numbered"].values,
    delta_df["delta_benzyl"].values
))

cefo_dict = dict(zip(
    delta_df["Feature, PDB numbered"].values,
    delta_df["delta_cefo"].values
))

project_pymol_top_features(
        per_feature_scores=benzyl_dict,
        model_name=f"benzyl",
        numb_features=100, 
        out_dir=rf"outputs/delta_special_general/"
    )

project_pymol_top_features(
        per_feature_scores=cefo_dict,
        model_name=f"cefotaxmine",
        numb_features=100, 
        out_dir=rf"outputs/delta_special_general/"
    )

The file: outputs/delta_special_general/benzyl_Pymol_Per_Feature_Scores.py was written to disk.
The file: outputs/delta_special_general/cefotaxmine_Pymol_Per_Feature_Scores.py was written to disk.
